In [37]:
import requests
import json
import pandas as pd
import numpy as np
from pandas import json_normalize

url_base = 'https://api.exactspotter.com/v3'
token = 'e4b1da77-06aa-4777-a594-cc5bb6e549c4'


headers = {
  'Content-Type': 'application/json',
  'token_exact': token
}

#Fazendo a consulta no endpoint Histórico de Qualificações - Listagem
response = requests.get(f'{url_base}/QualificationHistories', headers=headers)

#Jogando a response recebida em um dataframe para melhor visualização
df =  pd.DataFrame(response.json()['value'])

#Desaninhando a coluna user action, que mostra o funcionário responsável pela ação de qualificação, e separando em um df auxiliar. Depois, concateno esse df com o original
df_aux = pd.concat([json_normalize(x) for x in df['userAction']], ignore_index=True)
df  = pd.concat([df.drop('userAction', axis=1), df_aux], axis=1) 

#Tratando algumas colunas - unindo nome e sobrenome de usuário, tratando meetingDate nulo
df['user_full_name'] = df['name'] + ' ' + df['lastName']
df.rename(columns={
    'id': 'user_id', 
    'name': 'user_name', 
    'email': 'user_email'
}, inplace=True)

df['meetingDate'] = df['meetingDate'].replace("0001-01-01T00:00:00Z", np.nan)


#Dropando colunas não necessárias
colunas_de_interresse = ['leadId',
                    'stageId',
                    'stage',
                    'funnelId',
                    'score',
                    'qualificationDate',
                    'meetingDate',
                    'user_id',
                    'user_name',
                    'user_email',
                    'user_full_name'

]
df = df[colunas_de_interresse]
#Mudando tipos de dados para não precisar mudar no Power BI

#IDs são categorias, não números
df['leadId'] = df['leadId'].astype(str)
df['stageId'] = df['stageId'].astype(str)
df['funnelId'] = df['funnelId'].astype(str)

df['qualificationDate'] = pd.to_datetime(df['qualificationDate'], utc=True)
df['meetingDate'] = pd.to_datetime(df['meetingDate'], utc=True)


#Jogando para um csv que será importado via PowerBI
df.to_csv('dados_api.csv')


In [38]:
df.head()

,leadId,stageId,stage,funnelId,score,qualificationDate,meetingDate,user_id,user_name,user_email,user_full_name
0,16011372,5538,Filtro 1,620,Muito Quente,2023-08-18 12:14:07.035546800+00:00,NaT,275247,Carlos,csilva@exactsales.com.br,Carlos Silva
1,16011372,26089,Movimento Manual,620,Muito Quente,2023-08-18 12:14:19.089700900+00:00,NaT,275247,Carlos,csilva@exactsales.com.br,Carlos Silva
2,16693454,5538,Filtro 1,620,Muito Quente,2023-09-21 18:49:19.347632700+00:00,NaT,275247,Carlos,csilva@exactsales.com.br,Carlos Silva
3,16693454,26089,Movimento Manual,620,Muito Quente,2023-09-21 18:49:35.331407300+00:00,NaT,275247,Carlos,csilva@exactsales.com.br,Carlos Silva
4,16792134,5538,Filtro 1,620,Muito Quente,2023-10-03 13:53:59.912426+00:00,NaT,275247,Carlos,csilva@exactsales.com.br,Carlos Silva


In [27]:
from pandas import json_normalize
question_answers_df = pd.concat([json_normalize(x) for x in df['questionAnswers']], ignore_index=True)
question_answers_df['answers'][2]


# question_answers_df['answers'][0]

# # Lista para armazenar os DataFrames de cada linha
# answers_dfs = []

# # Iterar sobre cada item na coluna 'answers'
# for item in question_answers_df['answers']:
#     # Verificar se o item não é nulo ou NaN
#     if pd.notna(item):
#         # Converter o item JSON em um DataFrame e adicionar à lista
#         answers_dfs.append(json_normalize(item))
#     else:
#         # Adicionar um DataFrame vazio para manter a consistência
#         answers_dfs.append(pd.DataFrame())

# # Concatenar todos os DataFrames
# all_answers_df = pd.concat(answers_dfs, ignore_index=True)

[{'id': 66995341, 'position': None, 'text': 'Sim', 'active': False}]